## Librerias.

In [100]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

## Carga de datos

In [101]:
data_origin = pd.read_excel('Puntualidad Bogotá.xlsx')
datos = data_origin.copy()
datos.head(10)

#datos = datos[:100]

,Fecha / Hora,Asunto
0,2022-06-01T11:07:56+00:00,<7569|20220601> Shein - Control Horas
1,2022-06-01T11:25:34+00:00,<7865|20220601> Messer Consolidado Correos
2,2022-06-01T11:26:32+00:00,<6886|20220601> Messer Ocupación Agente
3,2022-06-01T11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio
4,2022-06-01T11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile
5,2022-06-01T11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...
6,2022-06-01T11:51:01+00:00,<6885|20220601> Messer 5/15
7,2022-06-01T12:00:26+00:00,<7745|20220601> Pichincha IVR
8,2022-06-01T12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...
9,2022-06-01T12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...


## Tipos de datos.

In [102]:
datos.dtypes

Fecha / Hora    object
Asunto          object
dtype: object

## Dimenciones del dataframe.

In [103]:
datos.shape

(5383, 2)

## Datos faltantes.

In [104]:
datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes

Fecha / Hora    0
Asunto          0
dtype: int64

## Porcentajes de datos faltantes por columna.

In [105]:
for column in datos.columns:
    porcentaje_faltantes = datos[column].isnull().mean()
    print(column, '=', round(porcentaje_faltantes*100,2))

Fecha / Hora = 0.0
Asunto = 0.0


## Convertir columna tipo object a datetime64 en una nueva columna.

In [106]:
datos['Fecha / Hora'] = pd.to_datetime(datos['Fecha / Hora'])

datos.dtypes

Fecha / Hora    datetime64[ns, UTC]
Asunto                       object
dtype: object

In [107]:
datos.head(10)

,Fecha / Hora,Asunto
0,2022-06-01 11:07:56+00:00,<7569|20220601> Shein - Control Horas
1,2022-06-01 11:25:34+00:00,<7865|20220601> Messer Consolidado Correos
2,2022-06-01 11:26:32+00:00,<6886|20220601> Messer Ocupación Agente
3,2022-06-01 11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio
4,2022-06-01 11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile
5,2022-06-01 11:37:53+00:00,<7776|20220601> BBVA - BBVA EMPRESAS Recuperac...
6,2022-06-01 11:51:01+00:00,<6885|20220601> Messer 5/15
7,2022-06-01 12:00:26+00:00,<7745|20220601> Pichincha IVR
8,2022-06-01 12:00:30+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...
9,2022-06-01 12:17:21+00:00,<6867|20220601> Cruz Verde Informe Diario Juni...


## Identificar los ID.

In [108]:
datos['ID'] = datos['Asunto'].str.extract('(\d{4})')
datos

,Fecha / Hora,Asunto,ID
0,2022-06-01 11:07:56+00:00,<7569|20220601> Shein - Control Horas,7569
1,2022-06-01 11:25:34+00:00,<7865|20220601> Messer Consolidado Correos,7865
2,2022-06-01 11:26:32+00:00,<6886|20220601> Messer Ocupación Agente,6886
3,2022-06-01 11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio,6887
4,2022-06-01 11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile,6887
...,...,...,...
5378,2022-06-30 20:00:10+00:00,<7408|20220630> BBVA - Reporte Tipificación Ou...,7408
5379,2022-06-30 20:00:46+00:00,<8061|20220630> Cruz Verde Tablero De Control ...,8061
5380,2022-06-30 20:01:10+00:00,<6874|20220630> Cruz Verde Actualización Kpi's...,6874
5381,2022-06-30 20:01:21+00:00,<7689|20220630> Medicarte Tablero De Control J...,7689


## Dato nulo identificado.

In [109]:
datos[589:611]

,Fecha / Hora,Asunto,ID
589,2022-06-03 11:20:59+00:00,<6885|20220603> Messer 5/15,6885
590,2022-06-03 11:38:06+00:00,ANDRES FELIPE PUERTO PINILLA <afpuertop@grupok...,NaN
591,2022-06-03 11:40:02+00:00,<7777|20220603> BBVA - BBVA EMPRESAS Recuperac...,7777
592,2022-06-03 11:41:21+00:00,<7019|20220603> Messer Respuesta De Correos Pa...,7019
593,2022-06-03 11:43:24+00:00,<7459|20220603> BBVA - Reporte Irene Seguros,7459
594,2022-06-03 11:43:24+00:00,<7459|20220603> BBVA - Reporte Irene Seguros,7459
595,2022-06-03 11:57:56+00:00,Re: <7749|20220601> Liberty Informes: GNB - Bo...,7749
596,2022-06-03 12:00:52+00:00,<6867|20220603> Cruz Verde Informe Diario Juni...,6867
597,2022-06-03 12:08:48+00:00,Seguimiento Tiendas <6238|20220603>,6238
598,2022-06-03 12:10:06+00:00,<6867|20220603> Cruz Verde Informe Diario Juni...,6867


## Identificando los datos faltantes por ID.

In [110]:
datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes

Fecha / Hora     0
Asunto           0
ID              17
dtype: int64

## Eliminar fila de ID faltante.

In [111]:

datos_faltantes = datos.isnull().sum().sort_values(ascending=True)
datos_faltantes
datos = datos[~(datos['ID'].isnull())]

## Porcentajes de ID faltantes

In [112]:
for column in datos.columns:
    porcentaje_faltantes = datos[column].isnull().mean()
    print(column, '=', round(porcentaje_faltantes*100,2))

Fecha / Hora = 0.0
Asunto = 0.0
ID = 0.0


## Verificamos la fina 590, datos faltante.

In [113]:
datos

,Fecha / Hora,Asunto,ID
0,2022-06-01 11:07:56+00:00,<7569|20220601> Shein - Control Horas,7569
1,2022-06-01 11:25:34+00:00,<7865|20220601> Messer Consolidado Correos,7865
2,2022-06-01 11:26:32+00:00,<6886|20220601> Messer Ocupación Agente,6886
3,2022-06-01 11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio,6887
4,2022-06-01 11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile,6887
...,...,...,...
5378,2022-06-30 20:00:10+00:00,<7408|20220630> BBVA - Reporte Tipificación Ou...,7408
5379,2022-06-30 20:00:46+00:00,<8061|20220630> Cruz Verde Tablero De Control ...,8061
5380,2022-06-30 20:01:10+00:00,<6874|20220630> Cruz Verde Actualización Kpi's...,6874
5381,2022-06-30 20:01:21+00:00,<7689|20220630> Medicarte Tablero De Control J...,7689


In [114]:
datos.shape

(5366, 3)

## Extraer comentario.

In [115]:
datos['Comment'] = "".join(re.findall("[a-z A-Z]",str(datos["Asunto"])))

/tmp/ipykernel_35153/902243950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['Comment'] = "".join(re.findall("[a-z A-Z]",str(datos["Asunto"])))


In [116]:
# datos['Comment'] = datos.apply(lambda x: "".join(re.findall("[a-z A-Z]",str(datos["Asunto"]))))
# datos.loc[i,column.Asunto] = datos['Comment']

In [117]:
extraer_comentario = lambda x: "".join(re.findall("[a-z A-Z]",str(x)))
datos['Comentario'] = datos['Asunto'].apply(extraer_comentario)

/tmp/ipykernel_35153/84422381.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['Comentario'] = datos['Asunto'].apply(extraer_comentario)


In [118]:
datos

,Fecha / Hora,Asunto,ID,Comment,Comentario
0,2022-06-01 11:07:56+00:00,<7569|20220601> Shein - Control Horas,7569,Shein Control Horas ...,Shein Control Horas
1,2022-06-01 11:25:34+00:00,<7865|20220601> Messer Consolidado Correos,7865,Shein Control Horas ...,Messer Consolidado Correos
2,2022-06-01 11:26:32+00:00,<6886|20220601> Messer Ocupación Agente,6886,Shein Control Horas ...,Messer Ocupacin Agente
3,2022-06-01 11:30:51+00:00,<6887|20220601> Messer Niveles De Servicio,6887,Shein Control Horas ...,Messer Niveles De Servicio
4,2022-06-01 11:33:21+00:00,<6887|20220601> Messer Niveles De Servicio Chile,6887,Shein Control Horas ...,Messer Niveles De Servicio Chile
...,...,...,...,...,...
5378,2022-06-30 20:00:10+00:00,<7408|20220630> BBVA - Reporte Tipificación Ou...,7408,Shein Control Horas ...,BBVA Reporte Tipificacin Outbound
5379,2022-06-30 20:00:46+00:00,<8061|20220630> Cruz Verde Tablero De Control ...,8061,Shein Control Horas ...,Cruz Verde Tablero De Control Unificado Junio
5380,2022-06-30 20:01:10+00:00,<6874|20220630> Cruz Verde Actualización Kpi's...,6874,Shein Control Horas ...,Cruz Verde Actualizacin Kpis Junio
5381,2022-06-30 20:01:21+00:00,<7689|20220630> Medicarte Tablero De Control J...,7689,Shein Control Horas ...,Medicarte Tablero De Control Junio
